<br>
<h1 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #C66363 ; color : #E8D6D8; text-align: center; border-radius: 100px 100px;">INTRODUCTION </h1>
<br>


* [Add Libaries](#1)
* [Load and Examine Data](#2)
* [Quick Look With Visualization](#3)
* [Dropping Outliers](#4)
* [Dropping & Fill Missing Values](#5) 
* [Feature Engineering](#6) 
* [Model Training](#7)

<a id="1"> </a>
# Add Libaries

In [ ]:
import numpy as np
import pandas as pd
import numpy as np # linear algebra
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings("ignore")

<a id="2"> </a>
# Load and Examine Data

In [ ]:
items_df = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
items_categ_df = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
shops_df = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
sales_train_df = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
test_df = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")

In [ ]:
items_df.head(3)

In [ ]:
items_categ_df.head(3)

In [ ]:
shops_df.head(3)

In [ ]:
sales_train_df.head(3)

In [ ]:
test_df.head(3)

<a id="6"> </a>
# Feature Engineering

### Creating Mounth Column

In [ ]:
mm=[]
for i in range(len(sales_train_df)):
    m = sales_train_df["date"].iloc[i].split(".")[1]
    mm.append(m)

sales_train_df["mounth"] = mm

### Group Items For Get Sell Item Per Mounth

In [ ]:
#pure_data = (sales_train_df.groupby(['mounth','item_id','shop_id'], as_index=False).sum()) 

In [ ]:
pure_data = sales_train_df.groupby(['mounth','item_id','shop_id'], as_index=False).sum()

In [ ]:
pure_data["spm"] = pure_data["item_cnt_day"] / 2.83

### Creating Sub Columns

In [ ]:
item_name = items_categ_df["item_category_name"]
items_categ_df["sub_cagetory"] = [i.split("-")[0].strip() for i in item_name]

prefix = shops_df["shop_name"]
shops_df["shop_pref"] = [p.split(" ")[0].strip() for p in prefix]



### Creating Mounth Column

In [ ]:
merge_data = pure_data.join(items_df.set_index('item_id'), on='item_id')
merge_data = merge_data.join(items_categ_df.set_index('item_category_id'), on='item_category_id')
merge_data = merge_data.join(shops_df.set_index('shop_id'), on='shop_id')

train_data = merge_data.drop(['shop_id', 'item_id' , 'item_category_id' , 'date_block_num'], axis=1)

In [ ]:
train_data.head()

In [ ]:
train_data = train_data.rename(columns={'item_cnt_day': 'all_sales'})

In [ ]:
test_df = test_df.join(items_df.set_index('item_id'), on='item_id')
test_df = test_df.join(items_categ_df.set_index('item_category_id'), on='item_category_id')
test_df = test_df.join(shops_df.set_index('shop_id'), on='shop_id')

test_df = test_df.drop(['shop_id', 'item_id' , 'item_category_id' ], axis=1)

In [ ]:
test_df.head(3)

In [ ]:
test_df['item_price'] = np.nan

In [ ]:
lowr = []
    
    
vc = train_data["sub_cagetory"].value_counts()
vc = pd.DataFrame(vc)
vc['name'] = vc.index
vc.reset_index(drop=True, inplace=True)

for i in range(len(vc)):
    
    if vc.iloc[i,0] < 2000:
        
        lowr.append(vc.iloc[i]["name"])
            
for i in range(len(train_data)):   
       
    if train_data["sub_cagetory"].iloc[i] in lowr:
            
        train_data["sub_cagetory"].iloc[i] = "other"

In [ ]:
lowr = []
    
    
vc = test_df["sub_cagetory"].value_counts()
vc = pd.DataFrame(vc)
vc['name'] = vc.index
vc.reset_index(drop=True, inplace=True)

for i in range(len(vc)):
    
    if vc.iloc[i,0] < 2000:
        
        lowr.append(vc.iloc[i]["name"])
            
for i in range(len(test_df)):   
       
    if test_df["sub_cagetory"].iloc[i] in lowr:
            
        test_df["sub_cagetory"].iloc[i] = "other"

<a id="3"> </a>
# Visualization Data

In [ ]:
train_data["item_name"].value_counts()

In [ ]:
train_data.info()

In [ ]:
train_data.describe()

In [ ]:
m=['01','02','03','04','05','06','07','08','09','10','11','12']

plt.figure(figsize=(15,10))
ax = sns.countplot(x = "mounth", data = train_data ,palette  = sns.cubehelix_palette(len(m)))
plt.xlabel('Mounth')
plt.ylabel('Counts')

In [ ]:
import plotly.graph_objects as go
vc=train_data["sub_cagetory"].value_counts()

labels = vc.index
values = vc.values

fig = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0, 0.1, 0])])
fig.show()

In [ ]:
f, ax = plt.subplots(figsize=(25, 10))
ax.tick_params(labelsize=15)
sns.scatterplot(data=train_data, x="spm", y="item_price", hue="sub_cagetory", palette="Paired")
f.subplots_adjust(top=0.9)
f.suptitle('SalePrice compared to YearBuilt, MSSubClass and TotalBsmtSF', fontsize="28");

<a id="3"> </a>
# Fill Test Data

In [ ]:
train_data.head(2)

In [ ]:
test_df.head(2)

In [ ]:
filling_data = train_data.copy()
filling_data = filling_data.drop(['mounth', 'all_sales' , 'item_price', 'item_name' ], axis=1)

y_fill = filling_data["spm"]
X_fill = filling_data.drop(['spm'], axis=1)

y_fill = pd.DataFrame(y_fill)
y_fill["spm"] = y_fill["spm"]


In [ ]:
X_fill = pd.get_dummies(X_fill, columns= ["item_category_name" , "sub_cagetory" , "shop_name" , "shop_pref"])

In [ ]:
from sklearn.model_selection import train_test_split
X_fill_train, X_fill_test, y_fill_train, y_fill_test = train_test_split(X_fill, y_fill, test_size=0.8, random_state=42)

In [ ]:
from sklearn import metrics


In [ ]:
X_fill_train.shape

In [ ]:
X_fill_train = np.array(X_fill_train)
y_fill_train = np.array(y_fill_train)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y_fill_train = ohe.fit_transform(y_fill_train).toarray()

In [ ]:

import keras
from keras.models import Sequential
from keras.layers import Dense
# Neural network
model = Sequential()
model.add(Dense(200, input_dim=187, activation='relu'))
model.add(Dense(140, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


#                metrics=['mae'

In [ ]:
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [ ]:
history = model.fit(X_fill_train, y_fill_train, epochs=100, batch_size=64)

In [ ]:
y_pred = model.predict(X_test)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))

In [ ]:
from sklearn.metrics import accuracy_score
a = accuracy_score(pred,test)
print('Accuracy is:', a*100)